### BentoML Example
# Titanic Survival Prediction with XGBoost

This is a BentoML Demo Project demonstrating how to package and serve XGBoost model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-tiantic-survival-prediction&dt=xgboost-tiantic-survival-prediction)

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install bentoml
!pip install xgboost numpy pandas

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [4]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   154k      0 --:--:-- --:--:-- --:--:--  154k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k


In [5]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [6]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()

,Pclass,Age,Fare,SibSp,Parch,Survived
0,3,22.0,7.2500,1,0,0
1,1,38.0,71.2833,1,0,1
2,3,26.0,7.9250,0,0,1
3,1,35.0,53.1000,1,0,1
4,3,35.0,8.0500,0,0,0


# Model Training

In [7]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

In [8]:
y_test =  model.predict(X_test)
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Create BentoService for model serving

In [13]:
%%writefile xgboost_titanic_bento_service.py

import xgboost as xgb

import bentoml
from bentoml.artifact import XgboostModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([XgboostModelArtifact('model')])
@bentoml.env(pip_dependencies=['xgboost'])
class TitanicSurvivalPredictionXgBoost(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler)
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

Overwriting xgboost_titanic_bento_service.py


# Save BentoML service archive

In [14]:
# 1) import the custom BentoService defined above
from xgboost_titanic_bento_service import TitanicSurvivalPredictionXgBoost

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionXgBoost()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-01-23 10:02:56,315] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-23 10:02:56,337] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.0
creating BentoML-0.6.0/BentoML.egg-info
creating BentoML-0.6.0/bentoml
creating BentoML-0.6.0/bentoml/artifact
creating BentoML-0.6.0/bentoml/bundler
creating BentoML-0.6.0/bentoml/cli
creating BentoML-0.6.0/bentoml/clipper
creating BentoML-0.6.0/bentoml/configuration
creating BentoML-0.6.0/bentoml/deployment
creating BentoML-0.6.0/bentoml/deployment/aws_lambda
creating BentoML-0.6.0/bentoml/deployment/sagemaker
creating BentoML-0.6.0/bentoml/handlers
creating BentoML-0.6.0/bentoml/migrations
creating BentoML-0.6.0/bentoml/migrations/versions
creating BentoML-0.6.0/bentoml/proto
creating BentoML-0.6.0/bentoml/repository
creating BentoML-0.6.0/bentoml/server
creating BentoML-0.6.0/bentoml/server/static
creating BentoML-0.6.0/bentoml/utils
creating BentoML-0.6.0/bentoml/utils/validator
creating BentoML-0.6.0/bentoml/yatai
creating BentoML-0.6.0/bentoml/yatai/client
copying files to BentoML-0.6.0...
copying LICENSE -> BentoML-0.6.0
copying MANIFEST.in -> BentoML-0.6

copying bentoml/utils/validator/__init__.py -> BentoML-0.6.0/bentoml/utils/validator
copying bentoml/yatai/__init__.py -> BentoML-0.6.0/bentoml/yatai
copying bentoml/yatai/deployment_utils.py -> BentoML-0.6.0/bentoml/yatai
copying bentoml/yatai/status.py -> BentoML-0.6.0/bentoml/yatai
copying bentoml/yatai/yatai_service_impl.py -> BentoML-0.6.0/bentoml/yatai
copying bentoml/yatai/client/__init__.py -> BentoML-0.6.0/bentoml/yatai/client
copying bentoml/yatai/client/bento_repository_api.py -> BentoML-0.6.0/bentoml/yatai/client
copying bentoml/yatai/client/deployment_api.py -> BentoML-0.6.0/bentoml/yatai/client
Writing BentoML-0.6.0/setup.cfg
UPDATING BentoML-0.6.0/bentoml/_version.py
set BentoML-0.6.0/bentoml/_version.py to '0.6.0'
Creating tar archive
removing 'BentoML-0.6.0' (and everything under it)
[2020-01-23 10:03:09,336] INFO - BentoService bundle 'TitanicSurvivalPredictionXgBoost:20200123100256_F85423' created at: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-t

## Load saved BentoService for serving


In [15]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

[2020-01-23 10:03:17,744] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-23 10:03:17,756] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
[2020-01-23 10:03:17,758] WARNING - Module `xgboost_titanic_bento_service` already loaded, using existing imported module.
[2020-01-23 10:03:17,762] WARNING - BentoML local changes detected - Local BentoML repository including 

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


# Work with BentoML CLI

**`bentoml get <BentoServiceName>` is great for list all versions of the BentoService**

In [16]:
!bentoml get TitanicSurvivalPredictionXgBoost

BENTO_SERVICE                                           CREATED_AT        APIS                       ARTIFACTS
TitanicSurvivalPredictionXgBoost:20200123100256_F85423  2020-01-23 18:03  predict::DataframeHandler  model::XgboostModelArtifact


**`bentoml get <BentoService name>:<BentoService version>` to access detailed information**

In [18]:
!bentoml get TitanicSurvivalPredictionXgBoost:20200123100256_F85423

{
  "name": "TitanicSurvivalPredictionXgBoost",
  "version": "20200123100256_F85423",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionXgBoost/20200123100256_F85423"
  },
  "bentoServiceMetadata": {
    "name": "TitanicSurvivalPredictionXgBoost",
    "version": "20200123100256_F85423",
    "createdAt": "2020-01-23T18:03:08.110800Z",
    "env": {
      "condaEnv": "name: bentoml-TitanicSurvivalPredictionXgBoost\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.5.8\nxgboost",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "XgboostModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API"
      }
    ]
  }
}


Use CLI to make predicition is easy. It's a great way to validate prediction result quickly

In [20]:
!bentoml run TitanicSurvivalPredictionXgBoost:20200123100256_F85423 predict --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]'

[2020-01-23 10:07:46,230] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.6.0
[0.46972126]


## Model Serving via REST API

In your termnial, run the following command to start the REST API server:

In [17]:
!bentoml serve {saved_path}

[2020-01-23 10:03:35,393] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.6.0
 * Serving Flask app "TitanicSurvivalPredictionXgBoost" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


Copy following command to make a curl request to Rest API server

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

# Containerize REST API server with Docker


The BentoService SavedBundle is structured to work as a docker build context, that can be directed used to build a docker image for API server. Simply use it as the docker build context directory:

In [25]:
!cd {saved_path} && docker build -t xgboost-titanic .

Sending build context to Docker daemon  1.009MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Running in 93c24cd29a40
Removing intermediate container 93c24cd29a40
 ---> 9396b863b5c4
Step 3/12 : EXPOSE 5000
 ---> Running in c7fd89ae56aa
Removing intermediate container c7fd89ae56aa
 ---> b30397557310
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Running in aadff73e4e57
+ apt-get update
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [174 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7908 kB]
Fetched 8318 kB in 2s

Preparing to unpack .../14-libitm1_8.3.0-6_amd64.deb ...
Unpacking libitm1:amd64 (8.3.0-6) ...
Selecting previously unselected package libatomic1:amd64.
Preparing to unpack .../15-libatomic1_8.3.0-6_amd64.deb ...
Unpacking libatomic1:amd64 (8.3.0-6) ...
Selecting previously unselected package libasan5:amd64.
Preparing to unpack .../16-libasan5_8.3.0-6_amd64.deb ...
Unpacking libasan5:amd64 (8.3.0-6) ...
Selecting previously unselected package liblsan0:amd64.
Preparing to unpack .../17-liblsan0_8.3.0-6_amd64.deb ...
Unpacking liblsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libtsan0:amd64.
Preparing to unpack .../18-libtsan0_8.3.0-6_amd64.deb ...
Unpacking libtsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libubsan1:amd64.
Preparing to unpack .../19-libubsan1_8.3.0-6_amd64.deb ...
Unpacking libubsan1:amd64 (8.3.0-6) ...
Selecting previously unselected package libmpx2:amd64.
Preparing to unpack .../20-libmpx2_8.3.0-6_amd64.deb ...
Unpacking libmpx2:a

conda-4.8.1          | 2.8 MB    | ########## | 100% 
certifi-2019.11.28   | 153 KB    | ########## | 100% 
numpy-base-1.18.1    | 4.2 MB    | ########## | 100% 
intel-openmp-2019.4  | 729 KB    | ########## | 100% 
mkl-2019.4           | 131.2 MB  | ########## | 100% 
mkl_fft-1.0.15       | 154 KB    | ########## | 100% 
mkl_random-1.1.0     | 321 KB    | ########## | 100% 
mkl-service-2.3.0    | 218 KB    | ########## | 100% 
scipy-1.3.2          | 13.9 MB   | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container 4ece37d063ef
 ---> af9be1b65cf4
Step 6/12 : COPY . /bento
 ---> 788ae6da74b6
Step 7/12 : WORKDIR /bento
 ---> Running in 0d27a02d4438
Removing intermediate container 0d27a02d4438
 ---> 311fd807f113
Step 8/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 23eac9b47e48
Removing intermediate container 23eac9b47e48
 -

  Created wheel for cerberus: filename=Cerberus-1.3.2-cp37-none-any.whl size=54336 sha256=5973e516f156897a190902d7ee12ec0bc6690841df13252860c41e43948829a9
  Stored in directory: /root/.cache/pip/wheels/e9/38/1f/f2cc84182676f3ae7134b9b2d744f9c235b24d2ddc8f7fe465
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp37-cp37m-linux_x86_64.whl size=1223694 sha256=36cd2bfd1cfc483675c6f0fab7600e55e68948bd3adc70438ac9691207f8ca91
  Stored in directory: /root/.cache/pip/wheels/b3/35/98/4c9cb3fd63d21d5606b972dd70643769745adf60e622467b71
  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23274 sha256=27ad315dffe1608cd32e548cebab77b37fb789c541cb5cc2d4071cd5088188e0
  Stored in directory: /root/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for alembic: filename=alembic-1.3.3-py2.py3-none-any.whl size=155684 sha256=9d92fe3f8627d153faa10667529b98c1536d8e3b8e74faebc189e080e5959c84
  Stored in directory: /root/.cache/pip/

  Created wheel for BentoML: filename=BentoML-0.6.0-cp37-none-any.whl size=505613 sha256=845d77f46f305a6018cbf1dd8dcc7a118f91ca442919fe8d595d2cc4435e603d
  Stored in directory: /root/.cache/pip/wheels/18/42/e5/7aade3a0ee2b7f6405b1751a9d08ca5d884cc501f6c2d72784
Successfully built BentoML
  Found existing installation: BentoML 0.5.8
    Uninstalling BentoML-0.5.8:
      Successfully uninstalled BentoML-0.5.8
Removing intermediate container df680189a88c
 ---> 736f91717cbd
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in 5575114fc642
Removing intermediate container 5575114fc642
 ---> 0462a3003826
Successfully built 0462a3003826
Successfully tagged xgboost-titanic:latest


Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [28]:
!docker run -p 5000:5000 xgboost-titanic

Traceback (most recent call last):
  File "/opt/conda/bin/bentoml", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/bentoml/cli/click_utils.py", line 94, in wrapper
    return func(*args, **kwargs)
TypeError: serve_gunicorn() got an unexpected keyword argument 'bundle_path'


# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For fastai pet classification, we are going to deploy to AWS Lambda

In [21]:
!bentoml lambda deploy first-xgboost-titanic \
    -b TitanicSurvivalPredictionXgBoost:20200123100256_F85423

Deploying Lambda deployment \[2020-01-23 10:09:56,484] INFO - Building lambda project
-[2020-01-23 10:14:15,686] INFO - Packaging AWS Lambda project at /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-cmmyid7z ...
|[2020-01-23 10:14:51,480] INFO - Deploying lambda project
\[2020-01-23 10:15:44,909] INFO - ApplyDeployment (first-xgboost-titanic, namespace dev) succeeded
Successfully created AWS Lambda deployment first-xgboost-titanic
{
  "namespace": "dev",
  "name": "first-xgboost-titanic",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionXgBoost",
    "bentoVersion": "20200123100256_F85423",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://8hsc80mck8.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-dev-first-xgboost-titanic-18bf13"


In [22]:
!bentoml lambda get first-xgboost-titanic

{
  "namespace": "dev",
  "name": "first-xgboost-titanic",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionXgBoost",
    "bentoVersion": "20200123100256_F85423",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://8hsc80mck8.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-dev-first-xgboost-titanic-18bf13"
    },
    "timestamp": "2020-01-23T18:20:36.476533Z"
  },
  "createdAt": "2020-01-23T18:09:51.245401Z",
  "lastUpdatedAt": "2020-01-23T18:09:51.245442Z"
}


**To send request to your AWS Lambda deployment, grab the endpoint URL from the json output above:**

In [23]:
!curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
https://8hsc80mck8.execute-api.us-west-2.amazonaws.com/Prod/predict














[0.469721257686615]

Use `bentoml lambda delete` to remove AWS Lambda deployment

In [24]:
!bentoml lambda delete first-xgboost-titanic

Successfully deleted AWS Lambda deployment "first-xgboost-titanic"
